<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Sentence_similarity/blob/main/Clustering_sb_Visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly
!pip install transformers[agents] datasets bertopic
!pip install -U sentence-transformers
!pip install einops
!pip install accelerate bitsandbytes
!pip install wordcloud
!pip install keybert
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.5 MB/s eta 0:00:0

In [2]:
from datasets import load_dataset
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import gradio as gr
from huggingface_hub import notebook_login

In [3]:
notebook_login()

#Gradio Demo App

In [4]:
dataset = load_dataset("Thabet/data_clustering_sb")

filtered_dataset = dataset.filter(lambda example: (example["week"] == 20) & (example["year"] == 2023) & (not example["Origin_sb"].startswith("RN")))

model_ckpt = "Thabet/all-mpnet-base-v2_for_sb_clustering"

embedding_model = SentenceTransformer(model_ckpt)

french_stopwords = list(STOP_WORDS) 
vectorizer_model = CountVectorizer(stop_words=french_stopwords, ngram_range=(1, 2),)

umap_model = UMAP(n_neighbors=15, n_components=20, metric='cosine', low_memory=False)

topic_model = BERTopic(embedding_model=embedding_model, vectorizer_model=vectorizer_model, umap_model=umap_model)
docs = filtered_dataset["train"]["Text_sb"]
topics, probs = topic_model.fit_transform(docs)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/125367 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Thabet___parquet/Thabet--data_clustering_sb-84bcff4e350e9dc0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Filter:   0%|          | 0/125367 [00:00<?, ? examples/s]

In [5]:
def intertopic_distance():
    fig=topic_model.visualize_topics()
    return fig

def document_distribution():
  fig = topic_model.visualize_documents(docs,)
  return fig

def create_wordcloud():
    topics =[]
    for topic_label, words in topic_model.get_topics().items():
      topics= topics + words
    text = {word: value for word, value in topics}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    fig = plt.figure(figsize=(8,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    return fig

def tree_map():
  values = list(topic_model.get_topic_info().Count)
  labels =  list(topic_model.get_topic_info().Name)

  # Create a trace for the treemap
  trace = go.Treemap(
      labels=labels,
      parents=["container"]*len(values),
      values=values,)

  # Create the layout
  layout = go.Layout(
      title="Treemap Plot",
      height=700,
      width = 1400)

  # Create the figure
  fig = go.Figure(data=[trace], layout=layout)
  return fig

def filter_dataset_on_topic(topic):
  filtered_dataset.set_format("pandas")
  df_dataset = filtered_dataset["train"][:]
  df_doc = topic_model.get_document_info(docs)
  df_topic = df_doc.loc[df_doc.Topic == topic][["Document", "Probability"]]
  df_topic=df_dataset.merge(df_topic,left_on="Text_sb", right_on="Document")[["Title_sb", "Origin_sb", "Redacteur_sb", "date_creation", "Probability"]]
  filtered_dataset.reset_format()
  return df_topic

In [6]:
with gr.Blocks() as demo:
  inputs = []
  # with gr.Row().style(equal_height=True):
  #   input1 = gr.Number(label="Week", )
  #   input2 = gr.Number(label="Year", )
  greet_btn = gr.Button("Click to view dashboard")
  with gr.Row().style(equal_height=True):
    output3 = gr.Plot()
    output1 = gr.Plot()
  output2 = gr.Plot() 
  output4 = gr.Plot()
  with gr.Row().style(equal_height=True):
    input = gr.Number(label="Topic", )
    sb_btn = gr.Button("list sb")
  output5 = gr.Dataframe()

  #greet_btn.click(fn=create_model, inputs=[input1,input2], outputs=[topic_model,filtered_dataset], api_name="greet")
  greet_btn.click(fn=intertopic_distance, inputs=inputs, outputs=output1, api_name="greet")
  greet_btn.click(fn=document_distribution, inputs=inputs, outputs=output2, api_name="greet")
  greet_btn.click(fn=create_wordcloud, inputs=inputs, outputs=output3, api_name="greet")
  greet_btn.click(fn=tree_map, inputs=inputs, outputs=output4, api_name="greet")
  sb_btn.click(fn=filter_dataset_on_topic, inputs=input, outputs=output5, api_name="greet")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>